# 머스크 트윗이 도지코인 가격에 미치는 영향 (Aggregated)
* 프로젝트의 background를 위한 분석
* 2021년 1월 ~ 4월의 18개 도지코인 관련 event를 aggregate

In [1]:
import os
from datetime import timedelta

from tqdm import tqdm

from data.price import get_price_data
from event import PriceAggregator, MUSK_TWEET_TIMES

aggregator = PriceAggregator()

In [2]:
# Event times
print(*MUSK_TWEET_TIMES, sep="\n")

2021-01-28 22:47:00
2021-02-04 07:35:00
2021-02-06 04:02:00
2021-02-07 07:41:00
2021-02-07 22:25:00
2021-02-10 15:08:00
2021-02-11 09:08:00
2021-02-14 23:25:00
2021-02-21 21:27:00
2021-02-24 13:00:00
2021-03-01 19:57:00
2021-03-06 04:40:00
2021-03-13 23:40:00
2021-04-01 11:25:00
2021-04-09 08:32:00
2021-04-15 04:33:00
2021-04-16 18:01:00
2021-04-28 07:20:00


### Create events without saving dataset
* 실행 후 바로 aggregate로 진행

In [3]:
aggregator.reset()
aggregator.create_multiple_events(
    event_times=MUSK_TWEET_TIMES,
    data_time_delta=timedelta(hours=10),
    event_window=(-50, +50),
    estimation_window=(-360, -60),
)

100%|██████████| 18/18 [00:19<00:00,  1.06s/it]


### Download dataset

In [ ]:
os.makedirs("price_dataset", exist_ok=True)
file_format = os.path.join("dataset", "price_{}.csv")

In [ ]:
data_timedelta = timedelta(days=1)

for idx, event_time in enumerate(tqdm(MUSK_TWEET_TIMES)):
    get_price_data(
        start_time=event_time - data_timedelta,
        end_time=event_time + data_timedelta,
        file_path=file_format.format(idx),
    )

### Create events with dataset files

In [ ]:
data_files = [file_format.format(i) for i in range(len(MUSK_TWEET_TIMES))]

aggregator.reset()
aggregator.create_multiple_events(
    event_times=MUSK_TWEET_TIMES,
    data_files=data_files,
    event_window=(0, +60),
    estimation_window=(-360, -60),
)

### Aggregate events

In [4]:
import os

os.makedirs("results", exist_ok=True)
result = aggregator.aggregate(result_file="results/price_aggr_m360_m60_5050.xlsx")